In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
import random
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from pywt import wavedec

2023-10-05 14:30:43.906163: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-05 14:30:43.944448: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-05 14:30:43.944478: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-05 14:30:43.944501: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-05 14:30:43.951876: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-05 14:30:43.952304: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
import os
dataset_path = '../Preprocessing/baru/'
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    for d in dirnames:
      for j, (dirpath2, dirnames2, filenames2) in enumerate(os.walk(os.path.join(dirpath, d))):
        for f in filenames2:
          print(os.path.join(dirpath2, f))

../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L5.csv
../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L7.csv
../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L8.csv
../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L1.csv
../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L2.csv
../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L9.csv
../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L4.csv
../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L10.csv
../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L3.csv
../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L6.csv
../Preprocessing/baru/db5/data_MfccDwtRmsCqtdb5L4.csv
../Preprocessing/baru/db5/data_MfccDwtRmsCqtdb5L6.csv
../Preprocessing/baru/db5/data_MfccDwtRmsCqtdb5L5.csv
../Preprocessing/baru/db5/data_MfccDwtRmsCqtdb5L8.csv
../Preprocessing/baru/db5/data_MfccDwtRmsCqtdb5L1.csv
../Preprocessing/baru/db5/data_MfccDwtRmsCqtdb5L9.csv
../Preprocessing/baru/db5/data_MfccDwtRmsCqtdb5L7.csv
../Preprocessing/baru/db5/data_MfccDwtRmsCqtdb5L3.csv
../Preprocessing/baru/db5/d

In [3]:
import os
dataset_path = '../Preprocessing/baru/'
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    for d in dirnames:
      for j, (dirpath2, dirnames2, filenames2) in enumerate(os.walk(os.path.join(dirpath, d))):
        for f in filenames2:
          file_path = os.path.join(dirpath2, f)
          print("Processing file: ", file_path)
          df = pd.read_csv(file_path)

          X = df.drop(['label', 'filename'], axis=1)
          y = df.iloc[:, -1]

          X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

          lb = LabelEncoder()
          X_train = np.expand_dims(X_train, axis=2)
          X_test = np.expand_dims(X_test, axis=2)
          y_test = tf.keras.utils.to_categorical(lb.fit_transform(y_test))
          y_train = tf.keras.utils.to_categorical(lb.fit_transform(y_train))

          model = tf.keras.Sequential([
              tf.keras.layers.SimpleRNN(64,activation='tanh',input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
              tf.keras.layers.Dense(32, activation="relu", activity_regularizer=tf.keras.regularizers.l2(1e-4)),

              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(64, activation="relu", activity_regularizer=tf.keras.regularizers.l2(1e-4)),
              tf.keras.layers.Dense(32, activation="relu", activity_regularizer=tf.keras.regularizers.l2(1e-4)),
              # tf.keras.layers.BatchNormalization(),

              tf.keras.layers.Dense(5, activation='softmax')
          ])

          optimazer = tf.keras.optimizers.Adam(learning_rate=0.0001)
          model.compile(optimizer=optimazer, loss='categorical_crossentropy', metrics=['acc'])

          history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

          test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

          to_append = f'{file_path}'
          to_append += f' {test_accuracy}'
          file = open('../Preprocessing/hasil.csv', 'a', newline='')
          with file:
              writer = csv.writer(file)
              writer.writerow(to_append.split())

Processing file:  ../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L5.csv
Epoch 1/50
20/20 [==============================] - 2s 34ms/step - loss: 1.5037 - acc: 0.3953 - val_loss: 1.3537 - val_acc: 0.6125
Epoch 2/50
20/20 [==============================] - 0s 13ms/step - loss: 1.2331 - acc: 0.6641 - val_loss: 1.1185 - val_acc: 0.7437
Epoch 3/50
20/20 [==============================] - 0s 13ms/step - loss: 0.9999 - acc: 0.7906 - val_loss: 0.9315 - val_acc: 0.7875
Epoch 4/50
20/20 [==============================] - 0s 13ms/step - loss: 0.8070 - acc: 0.8313 - val_loss: 0.8220 - val_acc: 0.7875
Epoch 5/50
20/20 [==============================] - 0s 13ms/step - loss: 0.6771 - acc: 0.8547 - val_loss: 0.7068 - val_acc: 0.8188
Epoch 6/50
20/20 [==============================] - 0s 13ms/step - loss: 0.5629 - acc: 0.8813 - val_loss: 0.6417 - val_acc: 0.8313
Epoch 7/50
20/20 [==============================] - 0s 13ms/step - loss: 0.4949 - acc: 0.9016 - val_loss: 0.5902 - val_acc: 0.8375
Epoch 8/50
